# Top Docs for Wikipedia 50-topics models

In [86]:
import sys
import os
import pathlib
import numpy as np
import json
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

if 'src.top_docs_selection.doc_selector' in sys.modules:
    print("doc_selector already in modules")
    del sys.modules['src.top_docs_selection.doc_selector']

from src.top_docs_selection.doc_selector import DocSelector

doc_selector already in modules


In [87]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [88]:
def load_model_info(model_path):
    if model_path.stem == "mallet":
        thetas = np.load(model_path / "doctopics.npz.npy")
        betas = np.load(model_path / "beta.npy")
    elif model_path.stem == "ctm":
        thetas = np.load(model_path / "train.theta.npy")
        betas = np.load(model_path / "beta.npy")
    else:
        print("Model type not recognized")
        return
    
    with open(model_path / "vocab.json") as infile:
        vocab_w2id = json.load(infile)
    
    bow = np.load(model_path / "bow_mat.npy")
    
    with (model_path / "topics.txt").open('r', encoding='utf8') as fin:
        keys = [el.strip() for el in fin.readlines()]
    
    return thetas, betas, vocab_w2id, bow, keys

def create_top_docs_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [df.text.iloc[top_docs[method][k][i]][0:1000] for k in range(len(thetas.T))] for i in range(5)}
    })

def create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [thetas.T[k][top_docs[method][k][i]] for k in range(len(thetas.T))] for i in range(5)}
    })

def reduce_threshold(matrix, initial_threshold, decrement=0.01):
    threshold = initial_threshold
    while True:
        modified_matrix = matrix.copy()
        satisfied = True

        for col in range(matrix.shape[1]):
            if np.all(matrix[:, col] <= threshold):
                satisfied = False
                break
        
        if satisfied:
            break
        
        threshold -= decrement

    return threshold

In [89]:
mallet_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/mallet")
ctm_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/ctm")
path_save_tops = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/top_docs")
corpus_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/train.metadata.jsonl")
text_column = "tokenized_text"

In [90]:
# Read documents
df = pd.read_json(corpus_path, lines=True)

df["id_top"] = range(len(df))
df["len"] = df[text_column].apply(lambda x: len(x.split()))
df["text_split"] = df[text_column].apply(lambda x: x.split())
corpus = df["text_split"].values.tolist()
#df.head()

In [91]:
methods = ['thetas', 'thetas_sample', 'thetas_thr']#'sall', 'spart', 's3'
n_top_words_s3 = 100
n_top_docs = 5

doc_selector = DocSelector()

## Mallet

In [92]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(mallet_path)

In [93]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 0.999596118927002 / 9.460147794015938e-07


In [94]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=mallet_path)
    for method in methods
}

In [95]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,0.998030,0.997481,0.997357,0.997273,0.997177
1,1,british ships french ship fleet spanish island captain admiral naval port coast sea battle crew squadron command navy sailed dutch war gun sent vessels arrived action later aboard men fire line took captured returned islands force royal_navy expedition convoy ordered august britain commander june frigate october sailing vessel merchant france,0.989845,0.986242,0.975379,0.974118,0.972367
2,2,horse horses white breed dog black breeds dogs breeding known animals century riding type bred small early animal arabian stud including population sheep named registered long old zoo coat american world years year stallion history today pony inches modern popular found united_states genetic breeders considered developed called pounds stakes racing,0.994985,0.982282,0.976747,0.929790,0.926588
3,3,match australia runs test innings england cricket team wickets scored second matches took australian played series season tour wicket class miller india ball bowling scoring run won final captain score batting century new_zealand day play harvey bowled victory bat tests win overs taking time victoria somerset playing average batsman players,0.996480,0.995988,0.986190,0.984264,0.982386
4,4,station line railway bridge trains new service construction built train services stations opened tunnel london class rail traffic lines use time route passenger system long road section motorway locomotives track transport passengers million oslo operated railways company platform power main cost work plans started branch south second closed locomotive north,0.995329,0.992506,0.986704,0.985714,0.983889
5,5,species shark long sharks females fish found birds small large males range bird brown white female fin known prey male black common tail length water breeding eggs size fins nest larger northern dorsal habitat population genus dark reef young southern year upper usually like teeth head including body great lower,0.998083,0.997636,0.997168,0.996253,0.995912
6,6,police said death prison murder case found trial people killed later evidence arrested reported court investigation crime according day told stated charges including time claimed report public days violence incident criminal group released guilty officers called arrest shot years drug december members victims february march media june body security men,0.824423,0.762204,0.756807,0.754640,0.744599
7,7,world won games olympics time olympic event round team final gold medal summer events record second athletes championships year open competition national seconds women place competed win winning murray contest men tournament meter sports sport silver finished held medals set heat finals august track international beijing lost cup bronze participated,0.995232,0.994828,0.994677,0.994189,0.994031
8,8,aircraft air flight wing squadron flying fighter pilot mission engine ground production crew pilots flew missions fuel radar landing range design service raf new combat control bomber test development june bomb fighters operational built speed operations system war program later engines bombers time september aerial nuclear high march october august,0.968833,0.963443,0.958630,0.952006,0.951871




####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,0.804800,0.899761,0.928135,0.944664,0.675716
1,1,british ships french ship fleet spanish island captain admiral naval port coast sea battle crew squadron command navy sailed dutch war gun sent vessels arrived action later aboard men fire line took captured returned islands force royal_navy expedition convoy ordered august britain commander june frigate october sailing vessel merchant france,0.986242,0.972367,0.928381,0.217794,0.061782
2,2,horse horses white breed dog black breeds dogs breeding known animals century riding type bred small early animal arabian stud including population sheep named registered long old zoo coat american world years year stallion history today pony inches modern popular found united_states genetic breeders considered developed called pounds stakes racing,0.730817,0.080983,0.843444,0.101013,0.048577
3,3,match australia runs test innings england cricket team wickets scored second matches took australian played series season tour wicket class miller india ball bowling scoring run won final captain score batting century new_zealand day play harvey bowled victory bat tests win overs taking time victoria somerset playing average batsman players,0.905316,0.922348,0.764739,0.622471,0.905316
4,4,station line railway bridge trains new service construction built train services stations opened tunnel london class rail traffic lines use time route passenger system long road section motorway locomotives track transport passengers million oslo operated railways company platform power main cost work plans started branch south second closed locomotive north,0.676023,0.937045,0.197647,0.757407,0.220984
5,5,species shark long sharks females fish found birds small large males range bird brown white female fin known prey male black common tail length water breeding eggs size fins nest larger northern dorsal habitat population genus dark reef young southern year upper usually like teeth head including body great lower,0.997636,0.779061,0.990459,0.727933,0.864504
6,6,police said death prison murder case found trial people killed later evidence arrested reported court investigation crime according day told stated charges including time claimed report public days violence incident criminal group released guilty officers called arrest shot years drug december members victims february march media june body security men,0.229877,0.200742,0.157735,0.096513,0.318535
7,7,world won games olympics time olympic event round team final gold medal summer events record second athletes championships year open competition national seconds women place competed win winning murray contest men tournament meter sports sport silver finished held medals set heat finals august track international beijing lost cup bronze participated,0.602413,0.933816,0.152675,0.375053,0.301382
8,8,aircraft air flight wing squadron flying fighter pilot mission engine ground production crew pilots flew missions fuel radar landing range design service raf new combat control bomber test development june bomb fighters operational built speed operations system war program later engines bombers time september aerial nuclear high march october august,0.883343,0.123342,0.276787,0.633110,0.884715




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,0.899761,0.896811,0.894165,0.893444,0.892741
1,1,british ships french ship fleet spanish island captain admiral naval port coast sea battle crew squadron command navy sailed dutch war gun sent vessels arrived action later aboard men fire line took captured returned islands force royal_navy expedition convoy ordered august britain commander june frigate october sailing vessel merchant france,0.893820,0.888176,0.879852,0.877769,0.875201
2,2,horse horses white breed dog black breeds dogs breeding known animals century riding type bred small early animal arabian stud including population sheep named registered long old zoo coat american world years year stallion history today pony inches modern popular found united_states genetic breeders considered developed called pounds stakes racing,0.897272,0.897051,0.891696,0.891580,0.891322
3,3,match australia runs test innings england cricket team wickets scored second matches took australian played series season tour wicket class miller india ball bowling scoring run won final captain score batting century new_zealand day play harvey bowled victory bat tests win overs taking time victoria somerset playing average batsman players,0.899576,0.893354,0.892173,0.891980,0.889254
4,4,station line railway bridge trains new service construction built train services stations opened tunnel london class rail traffic lines use time route passenger system long road section motorway locomotives track transport passengers million oslo operated railways company platform power main cost work plans started branch south second closed locomotive north,0.897884,0.897677,0.896002,0.894659,0.894405
5,5,species shark long sharks females fish found birds small large males range bird brown white female fin known prey male black common tail length water breeding eggs size fins nest larger northern dorsal habitat population genus dark reef young southern year upper usually like teeth head including body great lower,0.899338,0.898731,0.896419,0.896383,0.889896
6,6,police said death prison murder case found trial people killed later evidence arrested reported court investigation crime according day told stated charges including time claimed report public days violence incident criminal group released guilty officers called arrest shot years drug december members victims february march media june body security men,0.824423,0.762204,0.756807,0.754640,0.744599
7,7,world won games olympics time olympic event round team final gold medal summer events record second athletes championships year open competition national seconds women place competed win winning murray contest men tournament meter sports sport silver finished held medals set heat finals august track international beijing lost cup bronze participated,0.898961,0.898423,0.898331,0.897278,0.895956
8,8,aircraft air flight wing squadron flying fighter pilot mission engine ground production crew pilots flew missions fuel radar landing range design service raf new combat control bomber test development june bomb fighters operational built speed operations system war program later engines bombers time september aerial nuclear high march october august,0.898050,0.893859,0.893809,0.890589,0.887420


In [96]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,"Bremen-class cruiser = The Bremen class was a group of seven light cruisers built for the Imperial German Navy in the early 1900s . The seven ships , Bremen , Hamburg , Berlin , Lübeck , München , Leipzig , and Danzig , were an improvement upon the previous Gazelle class . They were significantly larger than the earlier class , and were faster and better armored . Like the Gazelles , they were armed with a main battery of ten 10.5 cm SK L / 40 guns and a pair of torpedo tubes . The ships of the Bremen class served in a variety of roles , from overseas cruiser to fleet scout to training ship . Bremen and Leipzig were deployed to the American and Asian stations , respectively , while the other five ships remained in German waters with the High Seas Fleet . At the outbreak of World War I in August 1914 , Leipzig was in the Pacific Ocean in the East Asia Squadron ; she saw action at the Battle of Coronel in November and was sunk a month later at the Battle of the Falkland Islands . Bremen","Italian battleship Emanuele Filiberto = The Emanuele Filiberto was a pre-dreadnought battleship built for the Italian Navy ( Italian : Regia Marina ) during the 1890s . Her keel was laid down in October 1893 and she was launched in September 1897 ; work was completed in April 1902 . She had one sister ship , Ammiraglio di Saint Bon , the lead ship of the Ammiraglio di Saint Bon class . She was armed with a main battery of four 10-inch ( 254 mm ) guns and was capable of a speed in excess of 18 knots ( 33 km / h ; 21 mph ) . Emanuele Filiberto served in the active squadron of the Italian navy for the first several years of her career . She was assigned to the 3rd Division during the Italo-Turkish War of 1911 – 1912 . During the war , she was involved in the assaults on Tripoli in North Africa and on the island of Rhodes in the eastern Mediterranean Sea . She was obsolescent by World War I and was slated to be broken up in 1914 – 15 , but the need for warships granted Emanuele Filiberto a","SMS Graudenz = SMS Graudenz was the lead ship of her class of light cruisers . She had one sister ship , SMS Regensburg . The ship was built by the German Kaiserliche Marine ( Imperial Navy ) in the Kaiserliche Werft shipyard in Kiel , laid down in 1912 and commissioned into the High Seas Fleet in August 1914 , days after the outbreak of World War I. She was named for the then-German town of Graudenz ( now Grudziądz , Poland ) . The ship was armed with a main battery of twelve 10.5 cm SK L / 45 guns and had a top speed of 27.5 knots ( 50.9 km / h ; 31.6 mph ) . Graudenz saw extensive service during World War I , including serving as part of the reconnaissance screen for the battlecruisers of the I Scouting Group during the raid on Scarborough , Hartlepool and Whitby in December 1914 . The ship also took part in the Battle of Dogger Bank in January 1915 , and the Battle of the Gulf of Riga in August 1915 . She had been damaged by a mine and was unable to participate in the Battle of Jut","Graudenz-class cruiser = The Graudenz class of light cruisers was a class of two ships built for the German Imperial Navy . The class comprised SMS Graudenz and SMS Regensburg . The ships both were laid down in 1912 , launched in October 1913 and April 1914 and commissioned in August 1914 and January 1915 , respectively . They were armed with a main battery of twelve 10.5 cm guns , though over the course of their careers , they were rearmed with seven more powerful 15 cm guns . They displaced 6,382 t ( 6,281 long tons ) at full load and were rated at a top speed of 27.5 knots (



####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,"SMS Yorck = SMS Yorck ( "" His Majesty 's Ship Yorck "" ) was the second and final ship of the Roon class of armored cruisers built for the German Imperial Navy . Yorck was named for Ludwig Yorck von Wartenburg , a Prussian field marshal . She was laid down in 1903 at the Blohm & Voss shipyard in Hamburg , and finished in November 1905 , at the cost of 16,241,000 marks . She displaced up to 9,875 metric tons ( 9,719 long tons ; 10,885 short tons ) and was armed with a main battery of four 21 cm ( 8.3 in ) guns . Her top speed was 20.4 knots ( 37.8 km / h ; 23.5 mph ) . The ship had a short career ; she served with the fleet for the first seven years , after which she was decommissioned and placed in reserve . After the outbreak of World War I , she was reactivated and returned to front-line service . After returning from the raid on Yarmouth on 3 – 4 November 1914 , the ship made a navigational error in heavy fog and accidentally sailed into a German defensive minefield . The ship sank q","Italian ironclad Sardegna = Sardegna was the third of three Re Umberto-class ironclad battleships built for the Italian Regia Marina ( Royal Navy ) . The ship , named for the island of Sardinia , was laid down in La Spezia in October 1885 , launched in September 1890 , and completed in February 1895 . She was armed with a main battery of four 13.5-inch ( 340 mm ) guns and had a top speed of 20.3 knots ( 37.6 km / h ; 23.4 mph ) — albeit at the cost of armor protection – and she was one of the first warships to be equipped with a wireless telegraph . Sardegna spent the first decade of her career in the Active Squadron of the Italian fleet . Thereafter , she was transferred to the Reserve Squadron , and by 1911 , she was part of the Training Division . She took part in the Italo-Turkish War of 1911 – 12 , where she escorted convoys to North Africa and supported Italian forces ashore by bombarding Ottoman troops . During World War I , Sardegna served as the flagship of the naval forces de","SMS Niobe = SMS Niobe ( "" His Majesty 's Ship Niobe "" ) was the second member of the ten-ship Gazelle class of light cruisers built by the Imperial German Navy . She was built by the AG Weser shipyard in Bremen , laid down in 1898 , launched in July 1899 , and commissioned into the High Seas Fleet in June 1900 . She was named after Niobe , a figure from Greek mythology . Armed with a main battery of ten 10.5 cm ( 4.1 in ) guns and two 45 cm ( 18 in ) torpedo tubes , Niobe was capable of a top speed of 21.5 knots ( 39.8 km / h ; 24.7 mph ) . The ship had a long career , serving in all three German navies , along with the Yugoslav and Italian fleets over the span of over 40 years in service . Niobe served in both home and overseas waters in the Imperial Navy , before being reduced to a coastal defense ship after the outbreak of World War I. She survived the conflict and was one of six cruisers permitted to the Reichsmarine by the Treaty of Versailles . In 1925 , the German Navy sold the","SMS Nürnberg ( 1916 ) = SMS Nürnberg was a Königsberg-class light cruiser built during World War I by Germany for the Imperial Navy . She had three sisters : Königsberg , Karlsruhe , and Emden . The ship was named after the previous light cruiser Nürnberg , which had been sunk at the Battle of the Falkland Islands . The new cruiser was laid down in 1915 at the AG Weser shipyard in Bremen , launched in April 1916 , and commissioned into the High Seas Fleet in February 1917 . Armed with eight 15 cm SK L / 45 guns , the ship had a top speed of 27.5 kn ( 50.9 km / h ; 31.6 mph 



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,ship ships war german fleet guns tons british torpedo long class cruiser service italian mph august naval submarine boat knots world boats cruisers speed september admiral main laid germany thick launched battery sank crew squadron april november served later built october armored short july training attack commissioned june sunk ordered,"Italian ironclad Sardegna = Sardegna was the third of three Re Umberto-class ironclad battleships built for the Italian Regia Marina ( Royal Navy ) . The ship , named for the island of Sardinia , was laid down in La Spezia in October 1885 , launched in September 1890 , and completed in February 1895 . She was armed with a main battery of four 13.5-inch ( 340 mm ) guns and had a top speed of 20.3 knots ( 37.6 km / h ; 23.4 mph ) — albeit at the cost of armor protection – and she was one of the first warships to be equipped with a wireless telegraph . Sardegna spent the first decade of her career in the Active Squadron of the Italian fleet . Thereafter , she was transferred to the Reserve Squadron , and by 1911 , she was part of the Training Division . She took part in the Italo-Turkish War of 1911 – 12 , where she escorted convoys to North Africa and supported Italian forces ashore by bombarding Ottoman troops . During World War I , Sardegna served as the flagship of the naval forces de","Italian battleship Napoli = Napoli was a Regina Elena-class pre-dreadnought battleship built for the Italian Regia Marina ( Royal Navy ) in 1903 – 08 . She was the last member of the four-ship class , which included the lead ship Regina Elena , Vittorio Emanuele , and Roma . Napoli was armed with a main battery of two 12-inch ( 305 mm ) and twelve 8 in ( 203 mm ) guns , and was capable of a top speed of 21 knots ( 39 km / h ; 24 mph ) . Napoli saw action in the Italo-Turkish War in 1911 and 1912 ; she took part in the attack on Derna , Libya , and the amphibious assaults on the islands of Rhodes and the Dodecanese in the Aegean Sea . Napoli remained in service during World War I in 1915 – 18 , but saw no action as a result of the cautious policies of both the Italian and Austro-Hungarian navies . She remained in the Italian inventory until she was stricken from the naval register in August 1926 and was subsequently broken up for scrap . = = Design = = Napoli was 144.6 meters ( 474 ft )","SM UB-3 = SM UB-3 was a German Type UB I submarine or U-boat in the German Imperial Navy ( German : Kaiserliche Marine ) during World War I. She disappeared on her first patrol in May 1915 , and was the first of her class to be lost . UB-3 was ordered in October 1914 and was laid down at the Germaniawerft shipyard in Kiel in November . UB-3 was a little more than 28 metres ( 92 ft ) in length and displaced between 127 and 142 metric tons ( 125 and 140 long tons ) , depending on whether surfaced or submerged . She carried two torpedoes for her two bow torpedo tubes and was also armed with a deck-mounted machine gun . She was launched and commissioned as SM UB-3 in March 1915 . UB-3 was broken into sections and shipped by rail to the Austro-Hungarian port of Pola in April for reassembly . She officially joined the Pola Flotilla on 1 May and departed on her first patrol for temporary duty in Turkey on 23 May , and was never seen again . A postwar German study concluded that UB-3 was likel","Italian cruiser Coatit = Coatit was a torpedo cruiser of the Italian Regia Marina built in the late 1890s . She was the second and final member of the Agordat class . The ship , which was armed with twelve 76 mm ( 3.0 in ) guns and two 450 mm ( 18 in ) torpedo tubes , was too slow and short-ranged to be able to scout effectively for the fleet , so her career was limited . She saw action during the Italo-Turkish War in 1911 – 12 , where she provided gunfire support to Italian troops in North Africa . She also caused a minor diplomatic incident from an attack on retreating Ottom

## CTM

In [97]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(ctm_path)

In [98]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

thr_list = list(thr)

thr_list[1] = reduce_threshold(thetas, thr_list[1], 0.05)
print("Updated thr_list:", thr_list)

# Adjust thr[0] to be at least 0.2 less than thr[1]
while thr_list[0] >= thr_list[1] - 0.2:
    thr_list[0] -= 0.01  
    if thr_list[0] <= 0:
        thr_list[0] = 0
        break

thr = tuple(thr_list)
print("Final thr:", thr)
print("Final thr[0]:", thr[0])

Max/min thetas: 0.9954009354114532 / 1.8919544385198605e-05
Updated thr_list: [0.5, 0.29999999999999977]
Final thr: (0.09999999999999969, 0.29999999999999977)
Final thr[0]: 0.09999999999999969


In [99]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=mallet_path)
    for method in methods
}

In [100]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,0.329688,0.326009,0.324521,0.320877,0.315103
1,1,film release released million story scene scenes script effects production opening plot filming original director sequel action movie weekend blu character bond highest batman score dvd photography cast based reviews films shot box grossing stars visual gave trailer screenplay disney version rotten_tomatoes marvel characters studio said office like rating set filmmakers sequels toy gross theaters wanted making footage burton idea animation critics worldwide animated direct theatrical cinematic shots grossed james_bond stated development horror sequence critical create principal cameron best special north_america halloween screening reception written escapes kills woody filmed villain animators cgi budget sequences movies role features place kill takes,0.501727,0.498330,0.490960,0.483859,0.473820
2,2,career hit games season league baseball major_league_baseball signed home played manager professional minor earned hits pitcher mlb leagues base giants roster strikeouts average pitched runs spent playing hitter starts spring era pitching basketball cubs inning team catcher national_league traded record starting rookie draft double named year run dodgers baseman batted contract disabled bats batters reds percentage hitting postseason pitchers rbi new_york_yankees seasons plate sophomore pirates prospect pitch rbis bases drafted pitches braves hitters walks ninth signing pennant finished player assists major boston_red_sox american_league chicago_cubs throwing philadelphia_phillies outfielder fastball handed batting fielder fielding shortstop texas_rangers freshman detroit_tigers tied san_francisco_giants list cincinnati_reds,0.737660,0.723641,0.714612,0.708765,0.705220
3,3,work life wrote writing published book written women works history world writer woman early books years known literary thought according children man people author novel poetry death art god human ideas society stories write poems literature historical story family age described writers modern century later church born writings study mother poem father science believed like social school culture poet view spiritual use religious tradition contemporary christian friend novels theory taught characters believe studies idea considered academic prize criticism old form account cult belief comics argued cultural professor great views political chess nature publication readers called publishing sexual teaching influenced philosophical,0.402178,0.376136,0.367494,0.365637,0.362293
4,4,tons beam deck consisted guns laid horsepower armor hull boilers displaced steam mounted armament battery knots armored protected stern inch belt commissioned aft scrap torpedo carried waterline ironclad muzzle ship tubes amidships italian shipyard assigned propulsion indicated thick nautical enlisted broken stricken millimeter conning armed trials turrets shaft launched flagship gun twin propeller fired pounder ironclads cruiser caliber submerged load bow driving engines sisters quick powered casemates armour coal forward supplied maximum turbines outbreak draft rounds mounts plates w



####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,0.069536,0.260127,0.209342,0.082293,0.012646
1,1,film release released million story scene scenes script effects production opening plot filming original director sequel action movie weekend blu character bond highest batman score dvd photography cast based reviews films shot box grossing stars visual gave trailer screenplay disney version rotten_tomatoes marvel characters studio said office like rating set filmmakers sequels toy gross theaters wanted making footage burton idea animation critics worldwide animated direct theatrical cinematic shots grossed james_bond stated development horror sequence critical create principal cameron best special north_america halloween screening reception written escapes kills woody filmed villain animators cgi budget sequences movies role features place kill takes,0.383667,0.018108,0.005598,0.010540,0.018520
2,2,career hit games season league baseball major_league_baseball signed home played manager professional minor earned hits pitcher mlb leagues base giants roster strikeouts average pitched runs spent playing hitter starts spring era pitching basketball cubs inning team catcher national_league traded record starting rookie draft double named year run dodgers baseman batted contract disabled bats batters reds percentage hitting postseason pitchers rbi new_york_yankees seasons plate sophomore pirates prospect pitch rbis bases drafted pitches braves hitters walks ninth signing pennant finished player assists major boston_red_sox american_league chicago_cubs throwing philadelphia_phillies outfielder fastball handed batting fielder fielding shortstop texas_rangers freshman detroit_tigers tied san_francisco_giants list cincinnati_reds,0.004719,0.008105,0.027764,0.094083,0.584125
3,3,work life wrote writing published book written women works history world writer woman early books years known literary thought according children man people author novel poetry death art god human ideas society stories write poems literature historical story family age described writers modern century later church born writings study mother poem father science believed like social school culture poet view spiritual use religious tradition contemporary christian friend novels theory taught characters believe studies idea considered academic prize criticism old form account cult belief comics argued cultural professor great views political chess nature publication readers called publishing sexual teaching influenced philosophical,0.131180,0.019885,0.006650,0.002442,0.163911
4,4,tons beam deck consisted guns laid horsepower armor hull boilers displaced steam mounted armament battery knots armored protected stern inch belt commissioned aft scrap torpedo carried waterline ironclad muzzle ship tubes amidships italian shipyard assigned propulsion indicated thick nautical enlisted broken stricken millimeter conning armed trials turrets shaft launched flagship gun twin propeller fired pounder ironclads cruiser caliber submerged load bow driving engines sisters quick powered casemates armour coal forward supplied maximum turbines outbreak draft rounds mounts plates w



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,0.298118,0.298068,0.297612,0.296216,0.295444
1,1,film release released million story scene scenes script effects production opening plot filming original director sequel action movie weekend blu character bond highest batman score dvd photography cast based reviews films shot box grossing stars visual gave trailer screenplay disney version rotten_tomatoes marvel characters studio said office like rating set filmmakers sequels toy gross theaters wanted making footage burton idea animation critics worldwide animated direct theatrical cinematic shots grossed james_bond stated development horror sequence critical create principal cameron best special north_america halloween screening reception written escapes kills woody filmed villain animators cgi budget sequences movies role features place kill takes,0.299881,0.298767,0.297957,0.296576,0.296187
2,2,career hit games season league baseball major_league_baseball signed home played manager professional minor earned hits pitcher mlb leagues base giants roster strikeouts average pitched runs spent playing hitter starts spring era pitching basketball cubs inning team catcher national_league traded record starting rookie draft double named year run dodgers baseman batted contract disabled bats batters reds percentage hitting postseason pitchers rbi new_york_yankees seasons plate sophomore pirates prospect pitch rbis bases drafted pitches braves hitters walks ninth signing pennant finished player assists major boston_red_sox american_league chicago_cubs throwing philadelphia_phillies outfielder fastball handed batting fielder fielding shortstop texas_rangers freshman detroit_tigers tied san_francisco_giants list cincinnati_reds,0.299455,0.296663,0.292269,0.291120,0.290753
3,3,work life wrote writing published book written women works history world writer woman early books years known literary thought according children man people author novel poetry death art god human ideas society stories write poems literature historical story family age described writers modern century later church born writings study mother poem father science believed like social school culture poet view spiritual use religious tradition contemporary christian friend novels theory taught characters believe studies idea considered academic prize criticism old form account cult belief comics argued cultural professor great views political chess nature publication readers called publishing sexual teaching influenced philosophical,0.297983,0.296374,0.295663,0.293647,0.292484
4,4,tons beam deck consisted guns laid horsepower armor hull boilers displaced steam mounted armament battery knots armored protected stern inch belt commissioned aft scrap torpedo carried waterline ironclad muzzle ship tubes amidships italian shipyard assigned propulsion indicated thick nautical enlisted broken stricken millimeter conning armed trials turrets shaft launched flagship gun twin propeller fired pounder ironclads cruiser caliber submerged load bow driving engines sisters quick powered casemates armour coal forward supplied maximum turbines outbreak draft rounds mounts plates w

In [101]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,"Oh Blue Christmas = Oh Blue Christmas is an EP by the American band A Fine Frenzy , released in November 2009 in the United States by Virgin Records . It was initially released exclusively through Target just two months following the group 's previous studio album Bomb in a Birdcage . The EP contains cover versions of three popular holiday songs ( "" Blue Christmas "" , "" Winter Wonderland "" and "" Christmas Time Is Here "" from A Charlie Brown Christmas ) as well as three original tracks . According to Alison Sudol , A Fine Frenzy 's leader , the collection of songs was recorded in five days at record producer David Bianco 's studio in Los Angeles . The EP , and "" Christmas Time Is Here "" in particular , received positive critical reception overall . In the United States , Oh Blue Christmas peaked at number 87 on the Billboard 200 , number three on Billboard 's Top Holiday Albums chart and number 28 on the Top Rock Albums chart . A Fine Frenzy was touring throughout the United States arou","The Stripped Mixes = The Stripped Mixes ( released on CD for a limited time as The Motown 50 Mixes ) , is a collection of American recording artist and former Jackson 5 member Michael Jackson 's classic songs . The songs featured on the album are mainly from Jackson 's career as a member of the Jackson 5 from the 1960s to the 1980s ; songs credited to Jackson as a solo artist are from his albums during his Motown-era . Other songs included on the album are "" stripped "" mixes of Jackson material , meaning the songs are of a quieter tone and most of the drums have been removed . Following a surge in Jackson 's popularity after his death in June 2009 , it was confirmed on July 7 , that The Stripped Mixes would be released . The album was made available as a digital download on July 7 , 2009 , and as a CD on July 28 . A remixed song , entitled "" I 'll Be There ( Minus Mix ) "" , was released on iTunes to promote the album on June 9 , 2009 , prior to Jackson 's death . The Stripped Mixes was","A Different Kind of Love Song = "" A Different Kind of Love Song "" is a song by American recording artist Cher , taken from her 24th studio album , Living Proof ( 2002 ) . The song was written and produced by Sigurd Rosnes and Johan Aberg , with additional writing done by Michelle Lewis , and was co-produced by Anders Hansson . The dance-pop song alludes to themes of tragedy , heroism and brotherhood , and was released as a double A-side single with "" The Music 's No Good Without You "" in July 2002 through Warner Bros. Records and WEA . "" A Different Kind of Love Song "" received mostly positive reviews from music critics , who deemed it as one of the album 's highlights , although noting Cher 's heavily processed vocals due to the use of auto-tune . The song charted on a few Billboard components , such as Dance / Club Play Songs , where it reached number one , Hot Dance Music / Maxi-Singles Sales , peaking at number two , and on the Adult Contemporary chart . No accompanying music video","Electric ( Robyn song ) = "" Electric "" is a song by Swedish recording artist Robyn from her sec



####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,"The Family Jewels ( Marina and the Diamonds album ) = The Family Jewels is the debut studio album recorded by Welsh singer Marina Diamandis , professionally known as Marina and the Diamonds . It was released on 15 February 2010 by 679 Recordings and Atlantic Records . Diamandis collaborated with several producers including Pascal Gabriel , Liam Howe , Greg Kurstin , Richard "" Biff "" Stannard , and Starsmith during its recording . She identifies the lyrical themes as "" the seduction of commercialism , modern social values , family and female sexuality . "" Contemporary music critics gave The Family Jewels fairly positive reviews , with the vocal delivery dividing opinions . The record debuted at number five on the UK Albums Chart with first-week sales of 27,618 copies . The album was eventually certified Gold by the British Phonographic Industry and has sold 195,358 units in the United Kingdom . The Family Jewels performed moderately on international record charts ; it peaked at number 1","Marilyn Monroe ( Nicki Minaj song ) = "" Marilyn Monroe "" is a song by Trinidadian born recording artist Nicki Minaj . The song was written by Minaj , Daniel James , Leah Haywood , Ross Golan and J.R. Rotem , while production was handled by J.R. Rotem and Dreamlab . Musically , "" Marilyn Monroe "" is an introspective mid-tempo ballad that contains an upbeat piano , synth beats , and features influences of bubblegum pop . Lyrically , the song alludes to oneself questioning the status of their relationship . The song makes frequent references to pop icon Marilyn Monroe , with many of her quotes woven into the song . "" Marilyn Monroe "" was generally well received by music critics , with some noting its crossover potential , while others felt like the song was too similar to other artists . The song has received comparisons to the 2011 singles "" Disaster "" by JoJo and "" Stuttering "" by Fefe Dobson , as well as Minaj 's own 2010 single "" Your Love "" from her debut album Pink Friday . The albu","Down Town = for the Greek magazine see Down Town ( magazine ) "" Down Town "" was the a 1987 release by The Justified Ancients of Mu Mu ( better known as The KLF ) . The song is gospel music driven by house music rhythms , incorporating a sample of Petula Clark 's 1964 single "" Downtown "" . = = Origins = = In 1987 , Bill Drummond and Jimmy Cauty formed The Justified Ancients of Mu Mu ( The JAMs ) , and busily released provocatively sample-heavy electronic music with beatbox rhythms and Drummond 's socially aware raps . Their debut single "" All You Need Is Love "" and album 1987 ( What the Fuck Is Going On ? ) were both investigated by the Mechanical Copyright Protection Society , who ordered The JAMs to recall and destroy all unsold copies of 1987 . A new single , "" Whitney Joins The JAMs "" , followed , along with a satirically edited version of the album , 1987 ( The JAMs 45 Edits ) , and the debut release from spinoff project Disco 2000 , "" I Gotta CD "" . By the time of the release of ""","The Manchester Rambler = "" The Manchester Rambler "" , also known as "" I 'm a Rambler 



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,credits composition allmusic listing charts piano drums beats liner label recording download ballad songwriter vocals albums musicians accompanying remix singer mixing singing adapted lyrically tempo formats lyrical pop lyrics acoustic billboard charted musically promotion performing mixed instrumentation background grammy videos promotional reception vocal personnel promote rolling_stone releases artist digital influences usher debuted sounds backing hip footage songwriting compilation guitars chart youtube listings handled popmatters bonus concert musician keyboards reviewer studio studios catchy percussion nme remixes airplay hop mexican vocalist guitarist band recordings thirty peaked tracks jazz sing collaboration rap electronic recorded bass hear highlight radio hot sings listeners dvd stan,"I 'm That Chick = "" I 'm That Chick "" is a song by American singer and songwriter Mariah Carey from her eleventh studio album , E = MC ² ( 2008 ) . A disco track with influences of R & B , it was written by Carey , Johntá Austin and production duo Stargate ( Mikkel S. Eriksen and Tor E. Hermansen ) . It contains elements of a song called "" Off the Wall "" originally performed by Michael Jackson . Rod Temperton , the songs composer , received a songwriting for "" I 'm That Chick "" as result . The track was positively received amongst most music critics , many of whom called it the best song on the album and praised is disco composition . Carey performed "" I 'm That Chick "" on The Hills season premiere party , the 2008 Fashion Rocks , Canadian Idol , and included it on the set-list of her eighth concert tour , The Elusive Chanteuse Show ( 2014 ) . It peaked at number 82 on the U.S. Billboard Hot R & B / Hip-Hop Songs chart . = = Recording and composition = = "" I 'm That Chick "" was written","Sing Like Me = "" Sing Like Me "" is a song by American recording artist Chris Brown . It was released as a promotional single from his third studio album , Graffiti , on November 24 , 2009 in the United States . The number was written by Brown , Big Makk , Keith Thomas , Lorenza "" Big Lo "" Lennon and Atozzio Towns , and produced by Makk , Thomas and Lennon . A pop and R & B ballad , the song contains lyrics about Brown leaving a nightclub with several women . "" Sing Like Me "" received mixed response from critics , and charted for two weeks on the US Hot R & B / Hip-Hop Songs in 2010 , peaking at number eighty-four . = = Background and composition = = "" Sing Like Me "" was written by Chris Brown , Big Makk , Keith Thomas , andf Lorenza "" Big Lo "" Lennon , with Makk , Thomas and Lennon producing the track . It was recorded at The Compound — a recording studio in Orlando , Florida — and Tony Maserati mixed the tune at The Record Plant — a studio in Los Angeles , California . The cover art o","Hot Tottie = "" Hot Tottie "" is a song by recording artist Usher . It was written by Usher , Ester Dean , Jay-Z and Polow da Don , with the latter producing it . The song features guest vocals from rapper Jay-Z and background vocals by Ester Dean . It is the second single in the United States and Canada from his EP , Versus , which is an extension of his sixth studio album , Raymond v. Raymond . The song was sent to rhythmic and urban airplay on August 9 , 2010 . "" Hot Tottie "" samples Big Tymers 's "" Big Ballin ' "" off their 1998 album How You Luv That Vol . 2 . "" Hot Tottie "" is an R & B song with hip hop tones , accompanied by strobing , electronic beats . It received positive reviews , with many critics noting it as a standout from the EP . It peaked at number twenty-one on the US Billboard Hot 100 , and was a top ten hit on the US Hot R & B / Hip-Hop Songs chart . Usher performed the song on The Early Show and on his OMG Tour . = = Background and composition = = The song was leaked","I 'm Not Your Hero = "" I 'm Not Your Hero "" is a song written , recorded and performe